# High-Performance Python
Once in awhile, you will hear that "oh Python is so slow or not performant enough" or "Python is not for <i>enterprise</i> use cases since it is not fast enough" or "Python is a language for script kiddies and at best "glue" code."  
ZOMG! Scala's functional API is awesome and stands on the shoulders of the JVM. Go(lang) has goroutines that beats the pants off of Python's multithreading. Rust--I don't know what it does but it sounds trendy. 🙄 

Spare me the faux concern ~~you pretentious C++ developer~~ ... I mean good question! Python supports rapid speed of development and experimentation, expressiveness and conciseness of syntax, and high-performance frameworks for speed. At a high-level, speed up Python performance on 1 machine using vectorization, multi-threading, multi-processing, and asynchronous programming. On multiple machines, you have cluster/distributed computing frameworks: Apache Spark, Dask, Apache Beam, Ray, and others. On cloud services, you have containers and scalable, serverless computing. In this tutorial, I will show you various ways to make Python fast and scalable, so performance is not your bottleneck.  

## Debunking the "Slow" Python Myth
What do you mean by slow? Do you mean that Python runs slow, so it would cost more money when renting EC2s?  
Remember that compute time is cheap and developer time is relatively expensive. Suppose a standard laptop has 4 CPUs and 16 GB of RAM. The equivalent EC2 instance on AWS rents for just \\$0.154 per hour; an engineer costs \\$50 per hour. So for each hour of developer time you waste, you could have rented a machine for 300 hours--or you can rent 300 machines for 1 hour. A 40 hour week for on EC2 would be \$6--cheaper than your daily lunch!  
Some "faster" languages are notoriously verbose, so you end up with pages and pages of code. Python's ease of use and expressivenes gives you a super power: rapid speed of development, which gives you a second super power: the ability to beat your competition to the market. Way of the Pro! Rapid experimentation allows you to out-maneuver your competition with better and newer features. Python's readability offers maintainability since developers come and go. Basically, can you write your code faster than somebody else: can you do something in 1 hour that somebody else takes 5?  

If you need speed, Guido recommends you delegate the performance critical part to Cython:  
```"At some point, you end up with one little piece of your system, as a whole, where you end up spending all your time. If you write that just as a sort of simple-minded Python loop, at some point you will see that that is the bottleneck in your system. It is usually much more effective to take that one piece and replace that one function or module with a little bit of code you wrote in C or C++ rather than rewriting your entire system in a faster language, because for most of what you're doing, the speed of the language is irrelevant." --Guido van Rossum```  
I won't mention Cython in the tutorial since it is rarely ever used--I have never personally met anybody who wrote their custom Cython code. Things that need to be performant in Python are already written in Cython: numpy, XGBoost, SpaCy, etc. The high level idea for Cython is that Python is actually written on top of C, so for speed you can write directly in C (or C++) and import it to Python.  

If you are concerned about speed, do you know what is faster than Python? Fortran.  Then you can brush up on your Fortran to get hired by ... nobody (or worse, mainframe specialists 🤢).  

Speed of the language primarily helps with CPU bound problems and not IO bound problems--in IO bound problems, any programming language will perform roughly the same since the majority of the time, you are waiting for a network response.  

Finally, you don't really care about the raw speed of a language. You care about the expressioness and functionality of the language: its ecosystem, user-base, and mindshare. Whatever problem you can think of, Python probably has a library for that. Which you can pip install. So you can write minimal code. With which you solve your problem. So you can call it a day. Like Newton, when you use Python, you stand on the shoulders of giants. Things just work! Imagine if you couldn't just import sklearn and run Random Forest but had to write your own random forest algorithm. Actually, don't...  
Here are some libraries just for scientific computing.  
<p align="center"><img src="images/scientific-ecosystem.png" width=600></p><br>  

In [1]:
import antigravity  # a cute Easter egg; things in Python are so easy. Python has "batteries included"

## The Big O, Not Just Japanese Batman
<p align="center"><img src="images/The-Big-O.jpg" width=300></p>

Data structures and algorithms (ie your functions) have different performance characteristics that can be measured by how much memory (space) it takes to do something and how many steps (time) it takes to do something. Fancy people use the terms <i>space and (run)time complexity</i>, respectively. Don't be intimidated by the fancy names; I assure you it's a simple idea.  
<b>Big O notation</b>: is used to denote the worst case scenario of how resource utilization (memory or number of steps) will grow with respect to the number of elements in your input data.  
Note: O(n) is pronounced oh-enn.

### Data Structure Complexity
Let's walk through some simple examples.  
list:
* to append to the end of a list, it is (an amoritized) O(1) operation. It is a fast operation.
* to append/insert to the beginning of a list, it takes O(n) operations because a new list is created, new element is inserted, and all the elements from the original list is copied over. It is slow in comparison to append to the right.
* to check if an element is in a list (ie test for membership), it takes O(n) operations since the worst case is that  element is not in the list, so you have to check every element of the list.
* to get an element at a known, specific ith index, it is an O(1) operation. Fast

dict:
* to check if a key is in a dictionary: O(1) operation because dict keys are hashable (and unique).
* to check if a value is in a dictinoary: O(n) operations because dict values are not hashable (and do not have to be unique).


Ideally all your code takes as little memory and as few steps as possible (ie O(1) for instantenous results). In practice, code uses non-trivial amounts of memory and computation. Sometimes, you can consider a tradeoff between space and time requirements. For example, if you are using a list and often have to append to the front and the end of the list, then you are better off with a data structure called double-ended queue (AKA deque). A deque have rapid `append()` and `appendleft()` methods as well as `pop()` and `popleft()` as these operations are all O(1). However, a deque has slow access to the ith element, which is now a O(n) operation. You have to figure out which data structure gives you the best benefit given tradeoffs or constraints.

### Algorithmic Complexity
As with data structures, algorithms also have space and time complexity. An algorithm is just a fancy word for a set of steps to get you to the right answer: sorting algorithm, sudoku solver algorithm, determining the winner in a tournament algorithm.

When an engineer says something is *(computationally) cheap*, that is fancy lingo for if the calculation uses little resources (low space or time complexity) and is fast. If a calculation is *(computationally) expensive*, it means it takes a lot of memory or is slow. Ideally you want your algorithm to be cheap.

There are orders of magnitudes for complexity of algorithms (and data structures):
* `O(1)` (AKA constant time): the best (that you can possibly do)!
* `O(log(n))` (AKA logarithmic time): good
* `O(n)` (AKA linear time): still good
* `O(n*log(n))`: fair, use sparingly
* `O(n^2)` (AKA quadratic time): bad, use very sparingly. If the exponent is any number other than 2 (ie 3, 4, ...), then it is called polynomial time
* `O(2^n)` (AKA exponential time) and `O(n!)`: horrible, ideally you never have use an algorithm (or data structure) that has these complexities because it will be slow and resource intensive.
    
Basically, you want your algorithm (or data structure) to *scale* well with the size of your input data. To visualize how important the scaling factor, Big O, is to your algorithm, here's a chart. Stay in the green.
<p align="center"><img src="images/Big_O_magnitudes.png" width=800></p><br>  
The chart and a more elaborate explanation of Big O is available at: <a href="https://towardsdatascience.com/understanding-time-complexity-with-python-examples-2bda6e8158a7">Understanding time complexity with Python examples</a>

Here is simple algorithm that concatenates strings together: it exhibits quadratic, O(n^2) time complexity.

In [1]:
from string import ascii_letters
from random import choices

print(ascii_letters)
lots_of_letters = choices(ascii_letters, k=100000)
print(lots_of_letters[:10])

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
['i', 'h', 'b', 's', 'v', 'm', 'U', 'q', 's', 'J']


In [2]:
%%time
# functional style: quadratic time
from functools import reduce

result_FP = reduce(lambda accumulated_string, char: accumulated_string + char, lots_of_letters)

Wall time: 171 ms


In [3]:
%%time
# procedural style: quadratic time
result_procedural = ""
for char in lots_of_letters:
    result_procedural += char

Wall time: 31 ms


Here's a smarter option that gives you linear, O(n) time complexity.

In [4]:
%%time
result_OOP = "".join(lots_of_letters)  # OOP is much faster since runtime is O(n)

Wall time: 1 ms


In [5]:
result_FP == result_procedural == result_OOP  # proof that the results are the same

True

"Smart data structures and dumb code works a lot better than the other way around." -from *The Cathedral and the Bazaar* by Eric S. Raymond

When possible, pick a good data structure that fits your problem--because there are a lot of data structures that do cool and useful things. Using the right data structure is the low-hanging fruit that requires minimum effort. If your logic is fundamentally complicated, then you will have to get creative and write (and optimize) a smart algorithm--that is more work. Of course, the quote is simply a rule of thumb--right most of the time, but not all the time. Use good judgment.

Let me give you a practical example. Often in technical interview questions, the interviewer will ask you to write/whiteboard an algorithm. This is an interview question I received many years ago when I was a beginner: "Given a list of words, find which words are anagrams of each other." Here is my very crude solution.

In [6]:
from collections import Counter

def find_anagrams(list_of_words):
    anagrams = {}
    for i, current_word in enumerate(list_of_words):
        for following_word in list_of_words[i+1:]:
            if Counter(current_word) == Counter(following_word):
                if current_word in anagrams:
                    anagrams[current_word].add(following_word)
                else:
                    anagrams[current_word] = {following_word}
    return anagrams

find_anagrams(["rats", "arts", "star", "god", "dog", "apple"])

{'rats': {'arts', 'star'}, 'arts': {'star'}, 'god': {'dog'}}

My algorithm is very inefficient since it has O(n^2) time complexity: it has a double for loop. And it is duplicating the letter counts (ie calling *Counter()*) multiple times for the same word. Finally, the output isn't great.

The correct way (Way of the Pro) is to find some way to transform the data into what I call a "canonical form" or "canonical representation" of the data that is invariant to differences that don't matter. For example, in this anagram problem, it does not matter what order the letters are in the word. Hence, to get the "canonical form", you sort the letters in the word.

In [7]:
from collections import defaultdict

def find_anagrams_optimized(list_of_words):
    anagrams = defaultdict(list)
    for word in list_of_words:
        word_with_letters_sorted = "".join(sorted(word))
        anagrams[word_with_letters_sorted].append(word)
    return [val for val in anagrams.values() if len(val) > 1]

find_anagrams_optimized(["rats", "arts", "star", "god", "dog", "apple"])

[['rats', 'arts', 'star'], ['god', 'dog']]

Notice that I used both:
* a better data structure: default dict that allows easy appending vs a regular dict
* a better algorithm: that has linear O(n) time complexity instead of O(n^2). No redundant, repeated computations

Moreover, the new function uses half the lines of code to get better results, so it is more readable and maintainable.

## Vectorized Calculations with SIMD
vectorization with numpy and pandas: SIMD  
like R's apply. pd.DataFrame.apply is not vectorized but without the speed performance. It's a convenience function.  
CPU vs GPU: ALU vs FPU  
order of magnitude of speed: CPU, RAM, storage, network: have image  

## Multi-threading vs Multi-processing
Multithreading and multiprocessing is when you hope to speed things up rather than wait for things to execute serially (ie a for loop). Multithreading is when you have multiple threads in the same process having access to the same variables and objects in memory--this is called *concurrency*. Multiprocessing is when you have multiple processes where each Python process/interpreter/session has its own memory and is running independently of each other--this is called true *parallelism*. Multithreading is faster to start up (<100 ms), but in most cases the right thing to do is use multiprocessing. Multithreading is used for *IO bound* tasks whereas multiprocessing is used for *CPU bound* tasks.
<table><tr>
    <td> <img src="images/concurrency.jpg" alt="Concurrency" style="width: 500px;"/> </td>
    <td> <img src="images/parallelism.jpg" alt="Parallelism" style="width: 500px;"/> </td>
</tr></table>
Beautiful visual diagrams from https://medium.com/rungo/achieving-concurrency-in-go-3f84cbf870ca

### Multithreading
Within 1 Python process, you can create multiple threads. Python has a Global Interpreter Lock (aptly shortened to GIL) that only allows 1 thread to be run at any given time, so you are switching between threads--this is called context switching.
<img src="images/multithreaded_process.png" alt="Context Switching Overhead" style="width: 300px;"/>
Multithreading is good for *IO bound* tasks. The typical example of IO bound task is scraping/downloading websites.
Suppose you have 100 urls you want to scrap and each url takes 1 second to scrap. Serially scraping the urls will take 100 seconds. However, if you multithread, IO tasks release the GIL, so you can scrap multiple urls "at the same time." Really, the bulk of the time is waiting for the response of data from the url, so multithreading can shrink that runtime significantly. Remember, you are still just using 1 CPU. Scraping websites does not take a lot of CPU power. Heavy number crunching is called *CPU bound* tasks, and multithreading does not help--and in fact, will be slower than serial execution.

A Python process uses 1 CPU, so each thread uses a *time slice* of the CPU. The thread that acquires the GIL gets to run for a few milliseconds, then is paused, and then releases the GIL. Another thread acquires the GIL, gets to run for a few milliseconds, then is paused, and then releases the GIL, etc. In practice, because only 1 thread is executed at any 1 time, multithreading is in some sense running "serially"--2 things are happening after each other, 2 things are NOT happening at exactly the same time. Imagine if you are the CPU and are multi-tasking multiple threads, you can be making a phone call, responding to an email, watching TV. You are switching between those threads very quickly (which are interweaved), so it gives the impression that you are doing them "at the same time."  
You can't know which thread is next, you can't know when the current thread is going to be paused, and you don't know which line of code the next thread will resume from. Well, you might think this is a bad idea--and you are correct.
* Newbie: multithreading sounds like a bad idea.  
* Intermediate: multithreading sounds like it could be good idea.  
* Pro: multithreading is definitely a bad idea!

In fact, multithreading is such a bad idea that Mozilla has this sign at their office. If you want to skip this section, feel free to do so. This section is to edify you on *not* using this technique.
<img src="images/be_this_tall_to_use_multithreading.jpg" alt="Don't use multithreading" style="width: 450px;"/> 



Why is multithreading a bad idea? Multiple threads have access to the same/shared objects in memory, so different threads can modify the shared object. The shared object can be corrupted due to the unexpected order of thread execution--this is called a **race condition**. The typical example is a bank transaction. Suppose Alice and Bob are 2 threads that share 1 bank account, and both want to withdraw money "at the same time" (within milliseconds of each other). Here's is the linearized timeline (since only 1 thread is running at any moment in time) of Alice and Bob triggering a race condition and corrupting the state of the shared object: the bank account balance. In blockchain, this is called the double spend problem.
* beginning bank account balance: \$100
* Alice: begins action of withdrawing \$60
* Alice: bank checks that balance (of \$100) >= \\$60: yes!
* Alice: ATM gives Alice \$60
* Bob: begins action of withdrawing \$70
* Bob: bank checks that balance (of \$100) >= \\$70: yes!
* Bob: ATM gives Bob \$70
* Alice: bank account decrements balance to \$40 (\\$100 - \\$60)
* Bob: bank account decrements balance to \$30 (\\$1000 - \\$70)
* ending bank account balance: \$30 (even though the bank should have prevented Bob from withdrawing money due to insufficient funds)


Race conditions can also occur with the opposite problem where if Alice and Bob are both depositing money "at the same time" (basically milliseconds of each other) as multiple threads.
* beginning bank account balance: \$100
* Alice: begins action of depositing \$60
* Alice: bank acccount gets current balance (\$100) and is about to increment
* Bob: begins action of depositing \$70
* Bob: bank account gets current balance (\$100) and is about to increment
* Bob: bank account increments balance to \$170 (\\$100 + \\$70)
* Alice: bank account increments balance to \$160 (\\$100 + \\$60)
* ending bank account balance: $160 (because it just so happens that Alice's thread finished last)

As you can see, multithreaded actions on a shared object are not "thread-safe", as the actions (ie the functions) are not atomic. Thread 1 might do something halfway and pause, and thread 2 can do something else on the same object. Not even functional programming can save you now. This problem would have not occurred if you just did things serially (ie a for loop). So why would you use multithreading instead of regular ol' serial execution?


The typical example of multithreading triggering a race condition is the bank transaction. Let's do something more fun: tug of war! Notice that the deque length does not always stay at 9,999,999 (deque length - 1 for the popped value) and the resulting deque has lost its order.

In [1]:
def popleft_and_appendright(dq, iterations, checkpoint_iterations):
    for i in range(iterations):
        value = dq.popleft()
        if i % checkpoint_iterations == 0:
            print("length of deque: {}".format(len(dq)))
        dq.append(value)
    print("final length of deque: {}".format(len(dq)))

def popright_and_appendleft(dq, iterations, checkpoint_iterations):
    for i in range(iterations):
        value = dq.pop()
        if i % checkpoint_iterations == 0:
            print("length of deque: {}".format(len(dq)))
        dq.appendleft(value)
    print("final length of deque: {}".format(len(dq)))

In [2]:
from collections import deque
from threading import Thread

# double-ended queue (deque) has O(1) operation for popleft/popright, appendleft/appendright
dq = deque(range(10000000))

thread1 = Thread(target=popleft_and_appendright, args=(dq, 10000000, 1000000))
thread2 = Thread(target=popright_and_appendleft, args=(dq, 10000000, 1000000))

thread1.start()
thread2.start()

thread1.join()
thread2.join()

length of deque: 9999999
length of deque: 9999999
length of deque: 9999998
length of deque: 9999999
length of deque: 9999998
length of deque: 9999999
length of deque: 9999999
length of deque: 9999999
length of deque: 9999999length of deque: 9999998

length of deque: 9999999
length of deque: 9999999
length of deque: 9999998
length of deque: 9999998
length of deque: 9999998length of deque: 9999998

length of deque: 9999998
length of deque: 9999998
length of deque: 9999998
length of deque: 9999999
final length of deque: 9999999
final length of deque: 10000000


In [3]:
from collections import Counter

print("Are all the elements in order? {}".format(Counter(x == y for x, y in zip(dq, range(len(dq))))))
print("first 10: {}".format(list(dq)[:10]))
print("last 10: {}".format(list(dq)[-10:]))

Are all the elements in order? Counter({True: 9697409, False: 302591})
first 10: [9999995, 2538, 9999997, 9999998, 28825, 0, 1, 2, 3, 4]
last 10: [9999985, 9999986, 9999987, 9999988, 9999989, 9999990, 9999991, 9999992, 9999993, 9999994]


Here's another example of the race condition: basically the bank account example. The bank account starts at 0 and should end at 0 but where it ends up, nobody knows! 🤔

In [4]:
def incrementer(lst, iterations, checkpoint_iterations):
    for i in range(iterations):
        lst[0] += 1
        if i % checkpoint_iterations == 0:
            print("lst: {}".format(lst))
    print("final lst: {}".format(lst))

def decrementer(lst, iterations, checkpoint_iterations):
    for i in range(iterations):
        lst[0] -= 1
        if i % checkpoint_iterations == 0:
            print("lst: {}".format(lst))
    print("final lst: {}".format(lst))

In [5]:
lst = [0]

thread1 = Thread(target=incrementer, args=(lst, 10000000, 1000000))
thread2 = Thread(target=decrementer, args=(lst, 10000000, 1000000))

thread1.start()
thread2.start()

thread1.join()
thread2.join()

lst: [1]lst: [0]

lst: [73496]
lst: [73664]
lst: [-72105]
lst: [-97734]
lst: [195419]
lst: [193173]
lst: [384526]
lst: [205554]
lst: [905035]
lst: [1116033]
lst: [1293557]
lst: [1409526]lst: [1378765]

lst: [1618449]
lst: [1668976]
lst: [1704706]
lst: [1539404]
final lst: [1678833]
lst: [1294087]
final lst: [294088]


The purpose of this example is that multithreading causes function calls to be "non-atomic". Multiple threads exist in the same memory and have access to the same variables. 2 (or more threads) compete to access and mutate variables. Suppose the first thread is executing some lines that update a list, that thread can be paused at any time in the function--and you have no idea where. Then a second thread can  be activated and then mutating that same variable. By the time, Python goes back to the first thread, the variable can already be changed. 

Finally, threads are *not* scalable. The scheduling of the threads (ie, which one comes next) is actually by the OS, so although a couple milliseconds seems fast to humans, it is still slow. Computers operate at the nanosecond level (billions of times per second). Switching between threads is called **context switching**. When there are a lot of threads (say 30), then the OS has to pick which thread out of the 30 is next. In this case, more time is spent acquiring the GIL than actually doing work. It's like the Hunger Games of threads--who gets to run next? In this case, it might be better to use serial execution (for loop) compared to using 30 threads.
<img src="images/context_switching_cost.jpg" alt="Context Switching Overhead" style="width: 500px;"/>

nonblocking  
only simple toy problems  
serial with so many locks  
cooperative vs competitive multitasking  
https://pybay.com/site_media/slides/raymond2017-keynote/async_examples.html

threads (concurrency) (race condition, tug of war) vs processes (parallel): concurrent.futures vs multiprocessing, shared vs independent memory  
concurrency vs parallelism vs asynchronous (event driven)  
give example of how the HBase was too slow and that Python was not the bottleneck--it was an IO problem    
My dirty little secret: ETLed 150 TB of data on 1 machine WITHOUT Spark  

## Asynchronous Programming/Event-Driven Programming
asyncio is in some sense serial with indirection of a coroutine. asynchronous immplemented in 3 ways: callback (hell from nested structure), futures/promises (looks like [flat] method chaining) and flatter, async/await  
yield from  
yield and return in the same function in a coroutine  
you don't need asyncio to do asynchronous programming  
C10k problem  
basically a queue--asynchronous because not executed immediately. The event loop is the buffer that manages queue  

## Distributed Computing Frameworks
A common lesson of high performance computing: High performance computing isn’t about doing one thing exceedingly well, it’s about doing nothing poorly.  
Spark, Dask (dataframe, bag, array, futures, delayed), Beam, Ray, 
amdahl's law  
vertical vs horizontal scaling  
In my opinion, distributed computing frameworks are glorified map/filter/reduce operations. Basically you want to chain a bunch of transformations together. If you have just 1 transformation, you will be better off with `multiprocessing`. You have a lot of elements (basically a distributed list) and then you want to do 1 thing to it, get an output, and then do another thing to that output. That's where functional programming comes in--how you approach the problem. The transformations are applied lazily, so they don't use memory until you want to materialize/trigger the actual computation.  

### MapReduce (old news)
explain what is MapReduce and disadvantages  
Out-of-core-computation (often mistakenly called out of memory computation): enables large matrix multiplications and querying large databases such as Hive query executed with MapReduce  
A `reduce` operation that is commutative and associative can be partially parallelized using a technique call <i>combiner</i>.  
Map reduce on-premise cannot be scalled up, so EMR on AWS gives you the ability to scale up when needed. Even Spark cluster is apportioned and probably isn't ideal for overallocation.  
show mapreduce stage picture  

### Apache Spark (industry standard 🏆)
If you want to get a high-demand skill to get a new job, learn Spark. I always say: `If you know Spark, you can't get fired!`  
Spark is basically MapReduce in memory. The bottleneck in large scale computation is getting the data to the CPU. RAM is way faster than disk; order of magnitudes, get picture of CPU cache, RAM, disk, network  
Simple Spark  
Spark and Dask: often a functional approach, functoolz syntax translates to Dask bags, FP allows embarrasingly parallel and laziness.  
Don't need Scala for speed  
Weird Java exceptions  
Show Simple-Spark  
Weakness: no ability to do multiple actions. The workaround is to split pipeline into 2 such as word count and character simultaneously and then do groupby.  
DataFrame, RDD
show Spark architecture picture  

### Dask (new framework with growing community adoption 🌱)
Distributed + Task  
`If you learn Dask, then you know Spark. And if you know Spark, you can't get fired!`  
Compare and contrast  
No weird Java exceptions  
Show Dask tutorial (tree aggregation example)  
delayed and futures  
x = x.map_blocks(compress).persist().map_blocks(decompress)  
Can do multiple actions  
DataFrame, array, bag, delayed
show Dask architecture picture (basically the same thing)  

### Apache Beam (theoretically interesting but nobody uses 😞)
explain why it is cool: unified API btween Batch + Stream, auto-scaling, serverless, templates, can do multiple actions  
pcollections, ParDo  
explain its weaknesses: does not support data with schema, no graph algorithms like K-Means, also no concept of action so have to use trick to trigger finality  
Throw picture in: 62 machines running for 30 minutes cost \$2.50.  

Spark, Dask, Beam  
Features:  
* **batch**: first classs, first class, first class
* stream: first class, kinda, first class
* **real time Dashboard (for debugging and throughput)**: yes, yes, yes
* **interactive mode/REPL**: yes, yes, yes
* **persist (memory and disk)**: memory+disk, only memory, no
* **with persist, can execute different sections independently**: yes, yes, no
* **can split 1 input to multiple output pipelines**: yes, yes, yes
* **simultaneous actions on multiple output pipelines**: no, yes, yes
* support for branching logic of pipeline: yes, yes, yes
* support for iterative algorithms: yes, yes, no
* **GROUP BY logic**: yes, yes, yes
* **JOIN logic**: yes, yes, yes
* **tagged output pipelines**: no, no, yes
* **combiner/aggregate logic**: yes, yes, yes
* **SQL support**: yes, kinda, no
* distributed numpy array: no, yes, no
* ML algorithms: yes, yes, no
* futures: no, yes, no
* delayed: no, yes, no
* support to address "hot keys": yes, yes, yes
* **shared variables**: broad variable and accumulators, no, idk
* supports logging for debugging: yes, yes, yes
* **supports multiple input/output file types**: yes, yes, yes
* click-based or GUI-based template: no, no, yes
* **auto scaling**: idk, idk, yes

Other support:
* integration: many, growing, primarily only GCP
* pure Python (3): no, yes, yes
* **easy to run/test on your own machine**: yes, yes, yes
* performant local cluster on your machine: idk, yes, no
* speed: Scala (fast), Python (less fast), Python (less fast)
* **documentation**: great, great, lacking
* user base: big, medium, low


Which one to use: Spark is a very hot data engineering skill. I tell my teammates: "if you know Spark, then you can't be fired." Since I work on Dask, I also tell my teammates: "if you know Dask, then you know Spark. If you know Spark, then you can't be fired."  


### Ray (and Actor Model)
like a blend of functional and OOP because it is stateful between transformations. Like Dask futures  
Plasma -> Apache Arrow    
totally novel architecture that overcomes IPC/RPC latency  
show Ray architecture picture  



These things are not built in a vacuum: progress stands on the shoulders of giants. Their histories are intertwined. Spark is basically mapreduce but in memory for the speed. Apache Beam originated from Google's internal Millwheel project and was marketed as true streaming. Spark basically copied Beam's approach. Dask was created originally for distributed arrays because Spark doesn't support it. Ray uses Redis and was probably inspired by Akka (framework in Scala). Dask adopted Ray's actor model. Basically copy ideas that work and dump the rest. A lot of frameworks that don't make the cut and just fade out. Other frameworks: Apache Apex (dead), Apache Gearpump (basically dead), Apache Samza (~~lame~~ only Java), Apache Flink (seems like a Hadoop thing, so basically for legacy applications)  

## Virtual Machines vs Containers
VM vs Docker/containerization, multi-tenancy  
serverless computing, scalable/decoupled, notifications and (asynchronous) PubSub connected to AWS Lambda (in some sense distributed but independent multiprocessing)  
world has moved on from hardware -> co-location -> virtualization (rented VM) -> container -> function: https://read.acloud.guru/the-evolution-from-servers-to-functions-21833b576744  
Focus on your business logic and delegate the rest to AWS. Who knows (and who cares) how its really implemented? Are you in the business of creating your private cloud/Hadoop or solving business problems that make $$$?  


## What is the Cloud?
There's an old saying: "Nobody gets fired for buying IBM". That was decades ago. I think the modern version would be "Nobody gets fired for buying AWS". The cloud is so versatile with all the services builtin, that if you are still buying hardware for your own datacenters without good reasons, you *should* be fired. One of the clients I worked on is such an old-boys club that they still buy hardware, explain story.  
AWS is really supply chain maximization, squeezing out efficiencies from elementary pieces: compute, storage, and networking. With these Lego blocks, they can create managed services that are just a copy of a Apache/open-source software and charge for it. The cloud doesn't cost you money--it saves you money. Overall win-win scenario. Like Costco or Trade Joe model 
Show all the logos of services from AWS or GCP. Show the open source version and then AWS service  

Why do you buy a car? Because it is better, faster, and/or cheaper than you can make it yourself. It just "works out of the box." You could build a car yourself; it would be worse functionality, slower to get to market, more expensive build from scratch, and a nightmare to maintain. Just buy it off-the-shelf. Focus on your main problem. Don't roll your own--you must be high if you do. Plus, if something breaks, there's somebody you can ~~complain to~~ ask for help.

## Extra Resources
Raymond Hettinger, Keynote on Concurrency, PyBay 2017: https://www.youtube.com/watch?v=9zinZmE3Ogk  
https://pybay.com/site_media/slides/raymond2017-keynote/threading.html  
About asynchornous programming from the creator of Twisted and key influencer to asyncio: https://glyph.twistedmatrix.com/2014/02/unyielding.html  